In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np
import tkinter as tk
from tkinter import filedialog, ttk
from PIL import Image, ImageTk
import threading

class AnimalClassifier(tf.Module):
    def __init__(self, model_path, labels):
        super().__init__()
        self.model = tf.saved_model.load(model_path)
        self.labels = labels

    def predict(self, image_array):
        # Reshape the input array to match the model input shape
        image_array = tf.reshape(image_array, (1, 224, 224, 3))

        # Make predictions using the model
        predictions = self.model.signatures['serving_default'](image_array)['output_0'][0]

        return predictions

class EmotionClassifier(tf.Module):
    def __init__(self, model_path, labels):
        super().__init__()
        self.model = tf.saved_model.load(model_path)
        self.labels = labels

    def predict(self, image_array):
        # Reshape the input array to match the model input shape
        image_array = tf.reshape(image_array, (1, 224, 224, 3))

        # Make predictions using the model
        predictions = self.model.signatures['serving_default'](image_array)['output_0'][0]

        return predictions
    
# Function to load and preprocess an image
def preprocess_image(image_path):
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return preprocess_input(img_array)

# Function to display result
def display_result(image_path, animal_classifier, emotion_classifier):
    print("Displaying result...")
    # Preprocess the uploaded image
    processed_image = preprocess_image(image_path)

    # Predict animal names and emotions with probabilities
    animal_predictions = animal_classifier.predict(processed_image)
    emotion_predictions = emotion_classifier.predict(processed_image)

    # Set probability thresholds
    animal_threshold = 0.04
    
    # Initialize lists to store detected animals and emotions
    detected_animals = []
    detected_emotions = []

    # Check animal predictions above threshold
    animal_indices = np.where(animal_predictions >= animal_threshold)[0]
    num_animals = len(animal_indices)  
    top_animal_indices = np.argsort(animal_predictions)[::-1][:num_animals]
    detected_animals = [animal_classifier.labels[i] for i in top_animal_indices]

    # Get top n emotions corresponding to the number of detected animals
    top_emotion_indices = np.argsort(emotion_predictions)[::-1][:num_animals]
    detected_emotions = [emotion_classifier.labels[i] for i in top_emotion_indices]

    # Load and display the image in the Tkinter window
    img = Image.open(image_path)
    img = img.resize((224, 224))  # Resize to match the model input size
    img_tk = ImageTk.PhotoImage(img)
    image_label.config(image=img_tk)
    image_label.image = img_tk

    # Update the result label with the detected animals and emotions
    result_label.config(text=f"Detected Animals: {', '.join(detected_animals)}\nDetected Emotions: {', '.join(detected_emotions)}")

def upload_and_process_image():
    # Open file dialog to select an image
    file_path = filedialog.askopenfilename()
    if file_path:
        print("Processing image...")
        # Update the progressbar to indicate processing
        progress_bar.start()

        # Create a new thread to run the processing task
        threading.Thread(target=process_image, args=(file_path,)).start()

def process_image(file_path):
    try:
        # Predict and display result
        display_result(file_path, animal_classifier, emotion_classifier)
    except Exception as e:
        print("Error:", e)

    # Update the progressbar to indicate processing is complete
    progress_bar.stop()

# Load animal classification model
animal_classifier = AnimalClassifier('Animal Classification', ['antelope', 'badger', 'bat', 'bear', 'bee', 'beetle', 'bison', 'boar', 'butterfly', 'cat',
                                                              'caterpillar', 'chimpanzee', 'cockroach', 'cow', 'coyote', 'crab', 'crow', 'deer', 'dog',
                                                              'dolphin', 'donkey', 'dragonfly', 'duck', 'eagle', 'elephant', 'flamingo', 'fly', 'fox',
                                                              'goat', 'goldfish', 'goose', 'gorilla', 'grasshopper', 'hamster', 'hare', 'hedgehog',
                                                              'hippopotamus', 'hornbill', 'horse', 'hummingbird', 'hyena', 'jellyfish', 'kangaroo', 'koala',
                                                              'ladybugs', 'leopard', 'lion', 'lizard', 'lobster', 'mosquito', 'moth', 'mouse', 'octopus',
                                                              'okapi', 'orangutan', 'otter', 'owl', 'ox', 'oyster', 'panda', 'parrot', 'pelecaniformes',
                                                              'penguin', 'pig', 'pigeon', 'porcupine', 'possum', 'raccoon', 'rat', 'reindeer', 'rhinoceros',
                                                              'sandpiper', 'seahorse', 'seal', 'shark', 'sheep', 'snake', 'sparrow', 'squid', 'squirrel',
                                                              'starfish', 'swan', 'tiger', 'turkey', 'turtle', 'whale', 'wolf', 'wombat', 'woodpecker', 'zebra'])

# Load emotion classification model
emotion_classifier = EmotionClassifier('Animal Emotion', ['happy', 'sad', 'hungry', 'other'])

# Create Tkinter window
window = tk.Tk()
window.title("Animal and Emotion Detector")

# Create Upload Button
upload_button = tk.Button(window, text="Upload Image", command=upload_and_process_image)
upload_button.pack(pady=10)

# Create Label to display image
image_label = tk.Label(window)
image_label.pack()

# Create a Label for displaying the result
result_label = tk.Label(window)
result_label.pack(pady=10)

# Create Progressbar to indicate processing
progress_bar = ttk.Progressbar(window, orient='horizontal', mode='indeterminate')
progress_bar.pack(pady=10)

# Run the Tkinter event loop
window.mainloop()

Processing image...
Displaying result...
Processing image...
Displaying result...
Processing image...
Displaying result...
Processing image...
Displaying result...
Processing image...
Displaying result...
